In [1]:
# Set me to the location of this directory...
%cd /Users/glenngalvizo/Documents/Graduate/cs-271a/cs271-sokoban/

/Users/glenngalvizo/Documents/Graduate/cs-271a/cs271-sokoban


In [2]:
import single_player_mcts as mc
import game_model as gm
import time
import json
import sqlite3
import datetime
import pandas
import os


input_file_prefix = 'resources/from_class/'  # We will report on the input files below.
input_files = [input_file_prefix + f for f in ['sokoban00.txt', 'sokoban02.txt', 'sokoban03.txt', 'sokoban01.txt']]
output_db = 'out/results.db'

default_mcts_params = {
    'heuristic_f': gm.Evaluation.heuristic_2,
    'simulation_bound': 300,
    'exploration_c': 0.5,
    'uncertainty_d': 50.0,
    'heuristic_correction_f': lambda h, root_h: h
}
default_iteration_count = 2000


# Create the output directory if it doesn't exist.
output_db_dir = output_db.split('/')[0]
if not os.path.exists(output_db_dir):
    os.makedirs(output_db_dir)


# Connect to our results database, and create the tables we need.
output_conn = sqlite3.connect(output_db)
output_cur = output_conn.cursor()
try:
    output_cur.execute("""
        CREATE TABLE Experiment (
            trial_id TEXT PRIMARY KEY,
            input_file TEXT,
            experiment TEXT
        );
    """)
    output_cur.execute("""
        CREATE TABLE Parameters (
            trial_id TEXT PRIMARY KEY,
            heuristic_f TEXT,
            simulation_bound INT,
            exploration_c FLOAT,
            uncertainty_d FLOAT,
            FOREIGN KEY (trial_id) REFERENCES Experiment (trial_id)
        );
    """)
    output_cur.execute("""
        CREATE TABLE Results (
            trial_id TEXT PRIMARY KEY,
            runtime FLOAT,
            iterations INT,
            solution_length INT,
            FOREIGN KEY (trial_id) REFERENCES Experiment (trial_id)
        );
    """)
    output_conn.commit()
except sqlite3.Error:
    pass  # There is no 'IF NOT EXISTS' support, so just "ask for forgiveness" here...


def perform_trial(input_file: str, mcts_params, experiment: str):
    # Build our game.
    root_state = gm.State.build(input_file)
    root_node = mc.Node(root_state)
    mcts_params['root'] = root_node
    x = mc.MCTS(**mcts_params)

    # Attempt to find a solution.
    starting_time = time.time()
    solution = x.run(default_iteration_count)
    end_time = time.time()

    # Setup the experiment and parameter table...
    primary_key = str(datetime.datetime.now())
    output_cur.execute(
        """ INSERT INTO Experiment VALUES (?, ?, ?); """,
        (primary_key, input_file, experiment, )
    )
    output_cur.execute(
        """ INSERT INTO Parameters VALUES  (?, ?, ?, ?, ?) """,
        (primary_key, str(mcts_params['heuristic_f']), mcts_params['simulation_bound'], mcts_params['exploration_c'],
         mcts_params['uncertainty_d'], )
    )

    # Save our results.
    if solution is not None:
        output_conn.execute(
            """ INSERT INTO Results VALUES (?, ?, ?, ?); """,
            (primary_key, end_time - starting_time, solution.number_of_iterations, len(solution.move_sequence))
        )
    else:
        output_conn.execute(
            """ INSERT INTO Results VALUES (?, ?, ?, ?); """,
            (primary_key, end_time - starting_time, None, None, )
        )


print('Running experiments with the following files: ')
print(json.dumps(input_files, indent=2))

Running experiments with the following files: 
[
  "resources/from_class/sokoban00.txt",
  "resources/from_class/sokoban02.txt",
  "resources/from_class/sokoban03.txt",
  "resources/from_class/sokoban01.txt"
]


In [3]:
# Setup plotting + analysis.
import mpld3
mpld3.enable_notebook()

import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.figsize'] = [9.5, 6]

# Heuristic Experiments Below

In [4]:
# Test which heuristic is best for our input files.
heuristic_functions = [gm.Evaluation.heuristic_1, gm.Evaluation.heuristic_2, gm.Evaluation.heuristic_3]
heuristic_corrections = [lambda h, root_h: h, lambda h, root_h: h, gm.Evaluation.heuristic_3_correction]
heuristic_labels = ['heuristic_1', 'heuristic_2', 'heuristic_3']


# Perform our experiment.
number_of_experiments = 10
for input_file in input_files:
    for f, c, label in zip(heuristic_functions, heuristic_corrections, heuristic_labels):
        for i in range(number_of_experiments):
            mcts_params = default_mcts_params.copy()
            mcts_params['heuristic_f'] = f
            mcts_params['heuristic_correction_f'] = c
            perform_trial(input_file, mcts_params, 'heuristic_f')

output_conn.commit()
print('Heuristic experiments are finished!')

Heuristic experiments are finished!


In [12]:
heuristic_f_data = pandas.read_sql_query(f"""
    SELECT A.heuristic_f, A.input_file, A.runtime, A.iterations, A.solution_length
    FROM (
        SELECT *
        FROM Experiment E, Parameters P, Results R
        WHERE E.trial_id = P.trial_id AND
              E.trial_id = R.trial_id AND
              E.experiment = 'heuristic_f'
        ORDER BY E.trial_id DESC
        LIMIT {number_of_experiments * len(input_files) * len(heuristic_functions)}
    ) AS A;
""", output_conn)

heuristic_f_data.groupby(['heuristic_f', 'input_file']) \
    .agg([np.mean, np.std, len])


runtime  \
                                                                                          mean   
heuristic_f                                      input_file                                      
<function Evaluation.heuristic_1 at 0x116721c20> resources/from_class/sokoban00.txt   0.002094   
                                                 resources/from_class/sokoban01.txt  61.666656   
                                                 resources/from_class/sokoban02.txt   0.009146   
                                                 resources/from_class/sokoban03.txt   0.339475   
<function Evaluation.heuristic_2 at 0x116721cb0> resources/from_class/sokoban00.txt   0.005728   
                                                 resources/from_class/sokoban01.txt  21.887089   
                                                 resources/from_class/sokoban02.txt   0.015388   
                                                 resources/from_class/sokoban03.txt   0.386674   
<function Evaluation.heuristic_3 at 0x116721d40> resources/from_class/sokoban00.txt   0.001132   
                                                 resources/from_class/sokoban01.txt  40.810361   
                                                 resources/from_class/sokoban02.txt   0.014869   
                                                 resources/from_class/sokoban03.txt   0.175204   

                                                                                                \
                                                                                           std   
heuristic_f                                      input_file                                      
<function Evaluation.heuristic_1 at 0x116721c20> resources/from_class/sokoban00.txt   0.003754   
                                                 resources/from_class/sokoban01.txt  47.299518   
                                                 resources/from_class/sokoban02.txt   0.006745   
                                                 resources/from_class/sokoban03.txt   0.151944   
<function Evaluation.heuristic_2 at 0x116721cb0> resources/from_class/sokoban00.txt   0.007336   
                                                 resources/from_class/sokoban01.txt  25.265652   
                                                 resources/from_class/sokoban02.txt   0.017263   
                                                 resources/from_class/sokoban03.txt   0.374557   
<function Evaluation.heuristic_3 at 0x116721d40> resources/from_class/sokoban00.txt   0.001130   
                                                 resources/from_class/sokoban01.txt  38.731302   
                                                 resources/from_class/sokoban02.txt   0.011656   
                                                 resources/from_class/sokoban03.txt   0.119021   

                                                                                           \
                                                                                      len   
heuristic_f                                      input_file                                 
<function Evaluation.heuristic_1 at 0x116721c20> resources/from_class/sokoban00.txt  10.0   
                                                 resources/from_class/sokoban01.txt  10.0   
                                                 resources/from_class/sokoban02.txt  10.0   
                                                 resources/from_class/sokoban03.txt  10.0   
<function Evaluation.heuristic_2 at 0x116721cb0> resources/from_class/sokoban00.txt  10.0   
                                                 resources/from_class/sokoban01.txt  10.0   
                                                 resources/from_class/sokoban02.txt  10.0   
                                                 resources/from_class/sokoban03.txt  10.0   
<function Evaluation.heuristic_3 at 0x116721d40> resources/from_class/sokoban00.txt  10.0   
                             

# Simulation Bound Experiments Below

In [6]:
# Perform our experiment.
simulation_bounds = [20, 50, 100, 200, 500, 1000]
number_of_experiments = 10
for input_file in input_files:
    for b in simulation_bounds:
        for i in range(number_of_experiments):
            mcts_params = default_mcts_params.copy()
            mcts_params['simulation_bound'] = b
            perform_trial(input_file, mcts_params, 'simulation_bound')

output_conn.commit()
print('Simulation bound experiments are finished!')

Simulation bound experiments are finished!


In [13]:
simulation_bound_data = pandas.read_sql_query(f"""
    SELECT A.simulation_bound, A.input_file, A.runtime, A.iterations, A.solution_length
    FROM (
        SELECT *
        FROM Experiment E, Parameters P, Results R
        WHERE E.trial_id = P.trial_id AND
              E.trial_id = R.trial_id AND
              E.experiment = 'simulation_bound'
        ORDER BY E.trial_id DESC
        LIMIT {number_of_experiments * len(input_files) * len(simulation_bounds)}
    ) AS A;
""", output_conn)

simulation_bound_data.groupby(['simulation_bound', 'input_file']) \
    .agg([np.mean, np.std, len])

runtime             \
                                                          mean        std   
simulation_bound input_file                                                 
20               resources/from_class/sokoban00.txt   0.000996   0.000593   
                 resources/from_class/sokoban01.txt  71.521710   9.003756   
                 resources/from_class/sokoban02.txt   0.010931   0.007637   
                 resources/from_class/sokoban03.txt   0.407440   0.597878   
50               resources/from_class/sokoban00.txt   0.000657   0.000102   
                 resources/from_class/sokoban01.txt  94.795253  15.703080   
                 resources/from_class/sokoban02.txt   0.008489   0.008465   
                 resources/from_class/sokoban03.txt   0.195287   0.271802   
100              resources/from_class/sokoban00.txt   0.000760   0.000234   
                 resources/from_class/sokoban01.txt  82.253291  54.308182   
                 resources/from_class/sokoban02.txt   0.007076   0.007349   
                 resources/from_class/sokoban03.txt   0.306863   0.323742   
200              resources/from_class/sokoban00.txt   0.000715   0.000224   
                 resources/from_class/sokoban01.txt  61.225107  41.280685   
                 resources/from_class/sokoban02.txt   0.010077   0.007123   
                 resources/from_class/sokoban03.txt   0.420657   0.254684   
500              resources/from_class/sokoban00.txt   0.000740   0.000327   
                 resources/from_class/sokoban01.txt  22.076819  23.783629   
                 resources/from_class/sokoban02.txt   0.005369   0.005025   
                 resources/from_class/sokoban03.txt   0.233206   0.147546   
1000             resources/from_class/sokoban00.txt   0.000664   0.000089   
                 resources/from_class/sokoban01.txt  39.225038  27.731726   
                 resources/from_class/sokoban02.txt   0.004108   0.003565   
                 resources/from_class/sokoban03.txt   0.403316   0.343882   

                                                          iterations  \
                                                      len       mean   
simulation_bound input_file                                            
20               resources/from_class/sokoban00.txt  10.0        1.0   
                 resources/from_class/sokoban01.txt  10.0        NaN   
                 resources/from_class/sokoban02.txt  10.0        1.8   
                 resources/from_class/sokoban03.txt  10.0       20.3   
50               resources/from_class/sokoban00.txt  10.0        1.0   
                 resources/from_class/sokoban01.txt  10.0     1027.0   
                 resources/from_class/sokoban02.txt  10.0        1.6   
                 resources/from_class/sokoban03.txt  10.0        6.0   
100              resources/from_class/sokoban00.txt  10.0        1.0   
                 resources/from_class/sokoban01.txt  10.0      571.0   
                 resources/from_class/sokoban02.txt  10.0        1.5   
                 resources/from_class/sokoban03.txt  10.0        7.8   
200              resources/from_class/sokoban00.txt  10.0        1.0   
                 resources/from_class/sokoban01.txt  10.0      836.7   
                 resources/from_class/sokoban02.txt  10.0        1.7   
                 resources/from_class/sokoban03.txt  10.0        8.9   
500              resources/from_class/sokoban00.txt  10.0        1.0   
                 resources/from_class/sokoban01.txt  10.0      263.7   
                 resources/from_class/sokoban02.txt  10.0        1.4   
                 resources/from_class/sokoban03.txt  10.0        5.9   
1000             resources/from_class/sokoban00.txt  10.0        1.0   
                 resources/from_class/sokoban01.txt  10.0      448.3   
                 resources/from_class/sokoban02.txt  10.0        1.1   
                 resources/from_class/sokoban03.txt  10.0        9.9   

                              

# Exploration C Experiments Below

In [8]:
# Perform our experiment.
exploration_c_terms = [0.25, 0.5, 0.75]
number_of_experiments = 10
for input_file in input_files:
    for c in exploration_c_terms:
        for i in range(number_of_experiments):
            mcts_params = default_mcts_params.copy()
            mcts_params['exploration_c'] = c
            perform_trial(input_file, mcts_params, 'exploration_c')

output_conn.commit()
print('Exploration C experiments are finished!')


Exploration C experiments are finished!


In [14]:
exploration_c_data = pandas.read_sql_query(f"""
    SELECT A.exploration_c, A.input_file, A.runtime, A.iterations, A.solution_length
    FROM (
        SELECT *
        FROM Experiment E, Parameters P, Results R
        WHERE E.trial_id = P.trial_id AND
              E.trial_id = R.trial_id AND
              E.experiment = 'exploration_c'
        ORDER BY E.trial_id DESC
        LIMIT {number_of_experiments * len(input_files) * len(exploration_c_terms)}
    ) AS A;
""", output_conn)

exploration_c_data.groupby(['exploration_c', 'input_file']) \
    .agg([np.mean, np.std, len])

runtime                   \
                                                       mean        std   len   
exploration_c input_file                                                       
0.25          resources/from_class/sokoban00.txt   0.001323   0.000419  10.0   
              resources/from_class/sokoban01.txt  45.413584  45.130188  10.0   
              resources/from_class/sokoban02.txt   0.006384   0.007437  10.0   
              resources/from_class/sokoban03.txt   0.249404   0.205114  10.0   
0.50          resources/from_class/sokoban00.txt   0.000888   0.000356  10.0   
              resources/from_class/sokoban01.txt  33.716427  42.631114  10.0   
              resources/from_class/sokoban02.txt   0.004949   0.005562  10.0   
              resources/from_class/sokoban03.txt   0.285758   0.239027  10.0   
0.75          resources/from_class/sokoban00.txt   0.000633   0.000099  10.0   
              resources/from_class/sokoban01.txt  31.778183  32.652755  10.0   
              resources/from_class/sokoban02.txt   0.006822   0.006901  10.0   
              resources/from_class/sokoban03.txt   0.226490   0.168692  10.0   

                                                  iterations              \
                                                        mean         std   
exploration_c input_file                                                   
0.25          resources/from_class/sokoban00.txt    1.000000    0.000000   
              resources/from_class/sokoban01.txt  596.900000  618.940214   
              resources/from_class/sokoban02.txt    1.200000    0.632456   
              resources/from_class/sokoban03.txt    7.400000    5.966574   
0.50          resources/from_class/sokoban00.txt    1.000000    0.000000   
              resources/from_class/sokoban01.txt  269.444444  131.959379   
              resources/from_class/sokoban02.txt    1.400000    0.843274   
              resources/from_class/sokoban03.txt    9.000000    6.798693   
0.75          resources/from_class/sokoban00.txt    1.000000    0.000000   
              resources/from_class/sokoban01.txt  412.600000  439.803037   
              resources/from_class/sokoban02.txt    1.500000    0.849837   
              resources/from_class/sokoban03.txt    7.200000    5.050853   

                                                       solution_length  \
                                                   len            mean   
exploration_c input_file                                                 
0.25          resources/from_class/sokoban00.txt  10.0        1.000000   
              resources/from_class/sokoban01.txt  10.0      150.300000   
              resources/from_class/sokoban02.txt  10.0        9.400000   
              resources/from_class/sokoban03.txt  10.0       41.400000   
0.50          resources/from_class/sokoban00.txt  10.0        1.000000   
              resources/from_class/sokoban01.txt  10.0      187.777778   
              resources/from_class/sokoban02.txt  10.0        6.600000   
              resources/from_class/sokoban03.txt  10.0       32.800000   
0.75          resources/from_class/sokoban00.txt  10.0        1.000000   
              resources/from_class/sokoban01.txt  10.0      210.800000   
              resources/from_class/sokoban02.txt  10.0        5.600000   
              resources/from_class/sokoban03.txt  10.0       29.800000   

                                                                   
                                                        std   len  
exploration_c input_file                                           
0.25          resources/from_class/sokoban00.txt   0.000000  10.0  
              resources/from_class/sokoban01.txt  67.410929  10.0  
              resources/from_class/sokoban02.txt   6.467010  10.0  
              resources/from_class/sokoban03.txt  19.196064  10.0  
0.50          resources/from_class/sokoban00.txt   0.000000  10.0  
              resources/from_class/sokoban01.txt  46.66

# Uncertainty D Experiments Below

In [10]:
# Perform our experiment.
uncertainty_d_terms = [25, 50, 100, 150]
number_of_experiments = 10
for input_file in input_files:
    for d in uncertainty_d_terms:
        for i in range(number_of_experiments):
            mcts_params = default_mcts_params.copy()
            mcts_params['uncertainty_d'] = d
            perform_trial(input_file, mcts_params, 'uncertainty_d')

output_conn.commit()
print('Uncertainty D experiments are finished!')

Uncertainty D experiments are finished!


In [15]:
uncertainty_d_data = pandas.read_sql_query(f"""
    SELECT A.uncertainty_d, A.input_file, A.runtime, A.iterations, A.solution_length
    FROM (
        SELECT *
        FROM Experiment E, Parameters P, Results R
        WHERE E.trial_id = P.trial_id AND
              E.trial_id = R.trial_id AND
              E.experiment = 'uncertainty_d'
        ORDER BY E.trial_id DESC
        LIMIT {number_of_experiments * len(input_files) * len(uncertainty_d_terms)}
    ) AS A;
""", output_conn)

uncertainty_d_data.groupby(['uncertainty_d', 'input_file']) \
    .agg([np.mean, np.std, len])

runtime                   \
                                                       mean        std   len   
uncertainty_d input_file                                                       
25.0          resources/from_class/sokoban00.txt   0.000747   0.000145  10.0   
              resources/from_class/sokoban01.txt  40.186787  52.307640  10.0   
              resources/from_class/sokoban02.txt   0.006170   0.006358  10.0   
              resources/from_class/sokoban03.txt   0.334238   0.338681  10.0   
50.0          resources/from_class/sokoban00.txt   0.000648   0.000116  10.0   
              resources/from_class/sokoban01.txt  38.284375  24.064942  10.0   
              resources/from_class/sokoban02.txt   0.011509   0.009831  10.0   
              resources/from_class/sokoban03.txt   0.233782   0.213907  10.0   
100.0         resources/from_class/sokoban00.txt   0.000550   0.000010  10.0   
              resources/from_class/sokoban01.txt  38.403032  32.016220  10.0   
              resources/from_class/sokoban02.txt   0.002990   0.003951  10.0   
              resources/from_class/sokoban03.txt   0.231698   0.228301  10.0   
150.0         resources/from_class/sokoban00.txt   0.000548   0.000008  10.0   
              resources/from_class/sokoban01.txt  44.769087  37.831063  10.0   
              resources/from_class/sokoban02.txt   0.006178   0.005535  10.0   
              resources/from_class/sokoban03.txt   0.199581   0.175518  10.0   

                                                  iterations              \
                                                        mean         std   
uncertainty_d input_file                                                   
25.0          resources/from_class/sokoban00.txt    1.000000    0.000000   
              resources/from_class/sokoban01.txt  305.222222  366.115685   
              resources/from_class/sokoban02.txt    1.600000    0.966092   
              resources/from_class/sokoban03.txt    9.700000   10.122033   
50.0          resources/from_class/sokoban00.txt    1.000000    0.000000   
              resources/from_class/sokoban01.txt  461.400000  298.532038   
              resources/from_class/sokoban02.txt    1.900000    1.286684   
              resources/from_class/sokoban03.txt    6.200000    5.452828   
100.0         resources/from_class/sokoban00.txt    1.000000    0.000000   
              resources/from_class/sokoban01.txt  489.900000  417.028763   
              resources/from_class/sokoban02.txt    1.200000    0.632456   
              resources/from_class/sokoban03.txt    6.700000    6.236986   
150.0         resources/from_class/sokoban00.txt    1.000000    0.000000   
              resources/from_class/sokoban01.txt  579.900000  506.684528   
              resources/from_class/sokoban02.txt    1.300000    0.674949   
              resources/from_class/sokoban03.txt    5.500000    4.624812   

                                                       solution_length  \
                                                   len            mean   
uncertainty_d input_file                                                 
25.0          resources/from_class/sokoban00.txt  10.0             1.0   
              resources/from_class/sokoban01.txt  10.0           194.0   
              resources/from_class/sokoban02.txt  10.0             4.4   
              resources/from_class/sokoban03.txt  10.0            26.7   
50.0          resources/from_class/sokoban00.txt  10.0             1.0   
              resources/from_class/sokoban01.txt  10.0           130.7   
              resources/from_class/sokoban02.txt  10.0            13.8   
              resources/from_class/sokoban03.txt  10.0            30.9   
100.0         resources/from_class/sokoban00.txt  10.0             1.0   
              resources/from_class/sokoban01.txt  10.0           171.0   
              resources/from_class/sokoban02.txt  10.0             4.6   
              resources/from_class/sokoban03.txt  